# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [3]:
licenca ="default"

In [4]:

ampl = tools.ampl_notebook(
    modules=["highs", "gcg", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


# 1- Base

Importando a base pré-processada com o limite de produção diário de:
<ul>
    <li>120.000 peças para o “Tipo de buffer” CX e </li>
    <li>100.000 peças para o “Tipo de buffer” EN.</li>
 </ul>

In [5]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process_revisado.xlsx')

df_process

,Unnamed: 0,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,CX,1865,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,CX,1865,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,CX,1865,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,CX,2489,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,CX,2988,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,EN,1863,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,EN,1863,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,EN,1863,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,EN,713,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707


In [6]:
df_process = df_process.rename(columns={'Unnamed: 0': "Id"})

In [7]:
df_process = df_process.rename(columns={'Capacidade': "Capacidade_rota"})

In [8]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [9]:
df_process['tempo_espera_dias'] = (df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16')

In [10]:
# Definir a função para calcular a prioridade
def calcular_prioridade(delta_t):
    diferenca = delta_t_maximo - delta_t
    if diferenca <= 5:
        return delta_t_maximo
    elif diferenca <= 10:
        return delta_t_maximo - 10
    else:
        return delta_t_maximo - 20

In [11]:
delta_t_maximo = df_process['tempo_espera_dias'].max()
df_process['prioridade'] = df_process['tempo_espera_dias'].apply(calcular_prioridade)
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


In [12]:
(df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16').head(300)

0      98
1      98
2      98
3      37
4      37
       ..
295    20
296    20
297    20
298    20
299    20
Name: Data_da_ordem_de_produção, Length: 300, dtype: int16

In [13]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [14]:
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


## Caixa padrão

In [15]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM_edit.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [16]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [17]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [18]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns=['Tipo','volume_util_caixa'], values=['Peças_max', 'volume_util_caixa']).reset_index()

Tipo_de_buffer  Peças_max                        \
Tipo                                Caixa 1    Caixa 2    Caixa 3   
volume_util_caixa                27778950.0 83393894.4 75538008.0   
0                             CX      170.0      170.0      170.0   
1                             EN      512.0      512.0      512.0   

                  volume_util_caixa                          
Tipo                        Caixa 1     Caixa 2     Caixa 3  
volume_util_caixa        27778950.0  83393894.4  75538008.0  
0                        27778950.0  83393894.4  75538008.0  
1                        27778950.0  83393894.4  75538008.0

In [19]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [20]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='Peças_max').add_prefix('Peças_max_').reset_index()

Tipo,Tipo_de_buffer,Peças_max_Caixa 1,Peças_max_Caixa 2,Peças_max_Caixa 3
0,CX,170,170,170
1,EN,512,512,512


In [21]:
df_caixas_tipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tipo               6 non-null      object 
 1   Tipo_de_buffer     6 non-null      object 
 2   Comprimento        6 non-null      int64  
 3   Largura            6 non-null      int64  
 4   Altura             6 non-null      int64  
 5   Fator_de_ocupação  6 non-null      float64
 6   Peças_max          6 non-null      int64  
 7   Peso_max_(kg)      6 non-null      int64  
 8   volume_util_caixa  6 non-null      float64
dtypes: float64(2), int64(5), object(2)
memory usage: 564.0+ bytes


In [22]:
df_caixas_tipo.columns

Index(['Tipo', 'Tipo_de_buffer', 'Comprimento', 'Largura', 'Altura',
       'Fator_de_ocupação', 'Peças_max', 'Peso_max_(kg)', 'volume_util_caixa'],
      dtype='object')

Temos um tipo de caixa padrão para cada tipo de buffer

In [23]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Levar a capacidade das caixas para cada ordem de produção:

O **número de peças máximo** varia com o buffer. Já o *volume* é igual para ambos os tipos, mudando apenas para o **tipo de caixa**

In [24]:
df_caixa_volume_max= df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

df_caixa_volume_max

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [25]:
df_process_capacitado = df_process.merge(df_caixa_volume_max, on='Tipo_de_buffer', how='inner')

df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,3,inf,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,1,inf,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0


In [26]:
df_process_capacitado.columns = df_process_capacitado.columns.str.replace(' ', '_')

In [27]:
import numpy as np
df_process_capacitado['volume_util_padrao'] = np.where(df_process_capacitado['Tipo_de_buffer'] == "EN", df_process_capacitado['volume_util_caixa_Caixa_3'], df_process_capacitado['volume_util_caixa_Caixa_2'])

In [28]:
df_process_capacitado = df_process_capacitado.merge(df_caixas_padrao[['Tipo_de_buffer', 'Peças_max']], on='Tipo_de_buffer', how='inner')
df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [29]:
df_process_capacitado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [30]:
df_process_capacitado['chave_loja_buffer_onda'].nunique()

1508

In [31]:
df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)

chave_loja_buffer_onda
10_EN_3    3459
28_EN_4    3187
16_EN_3    2093
2_EN_3     2086
31_EN_3    1802
33_EN_3    1786
8_EN_4     1620
10_EN_4    1509
9_EN_4     1464
3_EN_3     1403
25_EN_4    1367
18_EN_4    1360
48_EN_4    1355
8_EN_3     1348
10_EN_1    1329
23_EN_3    1299
11_EN_4    1228
3_EN_4     1162
2_EN_4     1156
9_EN_3     1127
Name: Peças, dtype: int64

In [32]:
(df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)).index

Index(['10_EN_3', '28_EN_4', '16_EN_3', '2_EN_3', '31_EN_3', '33_EN_3',
       '8_EN_4', '10_EN_4', '9_EN_4', '3_EN_3', '25_EN_4', '18_EN_4',
       '48_EN_4', '8_EN_3', '10_EN_1', '23_EN_3', '11_EN_4', '3_EN_4',
       '2_EN_4', '9_EN_3'],
      dtype='object', name='chave_loja_buffer_onda')

In [33]:
# Selecionar as n lojas maiores de um único buffer
n = 3
lojas_selecionadas_maiores = list((df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(n)).index)
lojas_selecionadas_maiores

['10_EN_3', '28_EN_4', '16_EN_3']

In [34]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
33800,33800,40,2023-10-30,847254655,443,233,32,0.303,10,CARTON_M,...,40_28.0,28_EN_4,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33801,33801,915,2023-10-30,847254621,445,242,33,0.323,10,CARTON_M,...,40_28.0,28_EN_4,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33802,33802,4051,2023-10-30,847254663,431,225,34,0.295,6,CARTON_M,...,40_28.0,28_EN_4,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33803,33803,8438,2023-10-30,847254639,432,248,32,0.310,12,CARTON_M,...,40_28.0,28_EN_4,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33804,33804,8788,2023-10-30,847254680,445,244,33,0.328,5,CARTON_M,...,40_28.0,28_EN_4,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,930_283.0,28_EN_4,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,930_283.0,28_EN_4,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,943_18.0,28_EN_4,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,943_18.0,28_EN_4,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [35]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")['Peças'].sum()

8739

In [36]:
df_process_capacitado["Volume_total"] = df_process_capacitado["Volume_unit"] * df_process_capacitado["Peças"]
df_process_capacitado["Peso_total"] = df_process_capacitado["Peso_unit"] * df_process_capacitado["Peças"]

In [37]:
(df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)).index

Index(['10_EN_3', '28_EN_4', '16_EN_3', '2_EN_3', '31_EN_3', '33_EN_3',
       '8_EN_4', '10_EN_4', '9_EN_4', '3_EN_3', '25_EN_4', '18_EN_4',
       '48_EN_4', '8_EN_3', '10_EN_1', '23_EN_3', '11_EN_4', '3_EN_4',
       '2_EN_4', '9_EN_3'],
      dtype='object', name='chave_loja_buffer_onda')

In [38]:
(df_process_capacitado.query('Tipo_de_buffer == "CX"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)).index

Index(['10_CX_1', '3_CX_9', '3_CX_1', '10_CX_4', '31_CX_1', '31_CX_9',
       '40_CX_9', '2_CX_9', '16_CX_6', '2_CX_4', '8_CX_4', '10_CX_9',
       '16_CX_9', '16_CX_1', '16_CX_4', '9_CX_4', '33_CX_9', '23_CX_9',
       '48_CX_4', '2_CX_1'],
      dtype='object', name='chave_loja_buffer_onda')

## Rotas

In [39]:
df_rotas = df_process_capacitado[['Rota', 'Capacidade_rota']].drop_duplicates()
df_rotas

,Rota,Capacidade_rota
0,1,inf
3,3,inf
9,10,2.720000e+10
14,8,4.000000e+10
26,6,2.720000e+10
45,13,2.720000e+10
71,11,2.720000e+10
76,12,1.600000e+10
116,5,2.720000e+10
147,15,4.000000e+10


# 2- Modelagem

## 2.1- HEURÍSTICA - FFD

First Fit Decreasing

In [40]:
import pandas as pd

def first_fit_decreasing(df, bin_restrictions, route_restrictions):
    # Sort items in decreasing order of volume and increasing order of date (FIFO)
    df_sorted = df.copy()
    df_sorted['Inv_Volume_total'] = (1 / df_sorted['Volume_total']).replace(np.inf, 0)
    df_sorted = df_sorted.sort_values(by=['Data_da_ordem_de_produção', 'Inv_Volume_total'], ascending=True)
    
    # Initialize bins list
    bins = []
    leftover_items = pd.DataFrame(columns=df_sorted.columns)  # Initialize leftover items DataFrame
    
    # Flag to track if a split was made
    split_made = True
    items_to_drop = []
    df_sorted = df_sorted.reset_index(drop=True)
    items_to_add = []

    while len(df_sorted) != 0:
        split_made = False
        df_sorted = df_sorted[~df_sorted.index.isin(items_to_drop)]

        # Add needed items to df_sorted
        if len(items_to_add) != 0:
            df_sorted = pd.concat([df_sorted, pd.DataFrame(items_to_add)])
        items_to_add = []
        df_sorted = df_sorted.reset_index(drop=True)
        items_to_drop = []
        
        # Make a copy of the sorted DataFrame
        df_sorted_copy = df_sorted.copy()
        
        # Iterate over items in the copy of the sorted DataFrame
        for index, item in df_sorted_copy.iterrows():
            # Flag to check if the item is placed
            placed = False
            
            # Try to place the item in existing bins
            for bin in bins:
                if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                    and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                    and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                    and item["Loja"] == bin["Loja"] \
                    and item["Rota"] == bin["Rota"] \
                    and item["Tipo_de_buffer"] == bin["Buffer"] \
                    and item["Classe_de_onda"] == bin["Wave"]:
                    
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']

                    # Remove the item from the original DataFrame
                    items_to_drop.append(index)
                    placed = True
                    break
            
            # If the item couldn't be placed entirely, reduce it by "inner" means until it fits into a bin
            while not placed and item['Peças'] > item['Inner']:
                # Reduce the item to its inner size
                detached_portion = item.copy()
                detached_portion['Peças'] = item['Inner']
                detached_portion['Volume_total'] = item['Volume_unit'] * item['Inner']
                detached_portion['Peso_total'] = item['Peso_unit'] * item['Inner']
                
                # Update the remaining portion of the item
                item['Peças'] -= item['Inner']
                item['Volume_total'] -= item['Volume_unit'] * item['Inner']
                item['Peso_total'] -= item['Peso_unit'] * item['Inner']
                
                # Concatenate the reduced portion back to the original DataFrame
                items_to_add.append(detached_portion)
                split_made = True
                
                # Try to place the reduced item in existing bins
                for bin in bins:
                    if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                        and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                        and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                        and item["Loja"] == bin["Loja"] \
                        and item["Rota"] == bin["Rota"] \
                        and item["Tipo_de_buffer"] == bin["Buffer"] \
                        and item["Classe_de_onda"] == bin["Wave"]:
                        
                        bin['Items'].append(index)  # Append index of the appended reduced item
                        bin['Volume'] += item['Volume_total']
                        bin['Peso_total'] += item['Peso_total']
                        bin['Amt.Items'] += item['Peças']

                        items_to_drop.append(index) # Remove the reduced item from the original DataFrame
                        placed = True
                        break
            
            # If the item still couldn't be placed, create a new bin
            if not placed:
                # Check if creating a new bin exceeds route capacity
                if len([bin for bin in bins if bin['Rota'] == item["Rota"]]) >= route_restrictions[route_restrictions["Rota"] == item["Rota"]]["Capacidade_rota"].values[0]:
                    leftover_items = leftover_items.append(item)  # Add item to leftover items DataFrame
                else:
                    bins.append({
                        'Items': [index],
                        'Volume': item['Volume_total'],
                        'Peso_total': item['Peso_total'],
                        'Amt.Items': item['Peças'],
                        "Loja": item["Loja"],
                        "Rota": item["Rota"],
                        "Buffer": item["Tipo_de_buffer"],
                        "Wave": item["Classe_de_onda"],
                        "vol_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["volume_util_caixa"].values[0],
                        "item_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["Peças_max"].values[0]
                    })
                    items_to_drop.append(index)
                    placed = True
                    break
    
    return bins, leftover_items


### 2.1.1- Situação 1

In [41]:
df_ffd_s1 = df_process_capacitado.query('chave_loja_buffer_onda in ("16_EN_3","9_EN_4","33_EN_3")')
df_ffd_s1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33868,33868,40,2023-10-30,847254655,443,233,32,0.303,3,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,9909024,0.909
33869,33869,915,2023-10-30,847254621,445,242,33,0.323,3,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10661310,0.969
33870,33870,4051,2023-10-30,847254663,431,225,34,0.295,2,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6594300,0.590
33871,33871,8788,2023-10-30,847254680,445,244,33,0.328,2,CARTON_M,...,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7166280,0.656
33872,33872,8438,2023-10-30,847254639,432,248,32,0.310,4,CARTON_M,...,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13713408,1.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59786,59786,1204,2023-11-21,879747286,321,146,45,0.240,5,CARTON_M,...,2108970,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10544850,1.200
59787,59787,4234,2023-11-21,879747315,309,156,65,0.203,2,CARTON_M,...,3133260,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6266520,0.406
59788,59788,8697,2023-11-21,879747294,314,134,41,0.230,7,CARTON_M,...,1725116,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12075812,1.610
59789,59789,9768,2023-11-21,879747307,320,147,42,0.217,3,CARTON_M,...,1975680,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5927040,0.651


In [42]:
df_ffd_s1['Peças'].sum()

5343

In [43]:
df_ffd_s1['Volume_total'].sum()

24368664108

In [44]:
bin_capacity_vol = 75538008
bin_capacity_weight = 23
bin_item_limit = 512
packed_bins, leftovers = first_fit_decreasing(df_ffd_s1, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")

Packed bins (new method):
Bin 1: Amt. Items 25 (Total Volume: 75520446 cubic mm | Total Weight: 7.7330kg)
Bin 2: Amt. Items 22 (Total Volume: 75051750 cubic mm | Total Weight: 7.0930kg)
Bin 3: Amt. Items 20 (Total Volume: 74832759 cubic mm | Total Weight: 4.3340kg)
Bin 4: Amt. Items 22 (Total Volume: 75289616 cubic mm | Total Weight: 4.7270kg)
Bin 5: Amt. Items 16 (Total Volume: 75334857 cubic mm | Total Weight: 3.6620kg)
Bin 6: Amt. Items 13 (Total Volume: 74927770 cubic mm | Total Weight: 3.8650kg)
Bin 7: Amt. Items 11 (Total Volume: 74927988 cubic mm | Total Weight: 3.7960kg)
Bin 8: Amt. Items 12 (Total Volume: 75484293 cubic mm | Total Weight: 4.7450kg)
Bin 9: Amt. Items 15 (Total Volume: 75473590 cubic mm | Total Weight: 3.5990kg)
Bin 10: Amt. Items 27 (Total Volume: 75426883 cubic mm | Total Weight: 5.4170kg)
Bin 11: Amt. Items 26 (Total Volume: 75298308 cubic mm | Total Weight: 5.6100kg)
Bin 12: Amt. Items 12 (Total Volume: 74798400 cubic mm | Total Weight: 4.7450kg)
Bin 13: Amt

### 2.1.2- Situação 2

In [45]:
df_ffd_s2 = df_process_capacitado.query("chave_loja_buffer_onda in ('10_EN_3', '28_EN_4', '2_EN_3')")

df_ffd_s2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33800,33800,40,2023-10-30,847254655,443,233,32,0.303,10,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,33030080,3.030
33801,33801,915,2023-10-30,847254621,445,242,33,0.323,10,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,35537700,3.230
33802,33802,4051,2023-10-30,847254663,431,225,34,0.295,6,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,19782900,1.770
33803,33803,8438,2023-10-30,847254639,432,248,32,0.310,12,CARTON_M,...,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,41140224,3.720
33804,33804,8788,2023-10-30,847254680,445,244,33,0.328,5,CARTON_M,...,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17915700,1.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,45989424,5.160
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,28047600,2.805
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6334200,0.574
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5670000,0.614


In [46]:
df_ffd_s2['Peças'].sum()

8732

In [47]:
df_ffd_s2['Volume_total'].sum()

42046147044

In [48]:
bin_capacity_vol = 75538008
bin_capacity_weight = 23
bin_item_limit = 512
packed_bins, leftovers = first_fit_decreasing(df_ffd_s2, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")

Packed bins (new method):
Bin 1: Amt. Items 22 (Total Volume: 75473176 cubic mm | Total Weight: 6.8740kg)
Bin 2: Amt. Items 24 (Total Volume: 74629846 cubic mm | Total Weight: 8.2840kg)
Bin 3: Amt. Items 20 (Total Volume: 73823495 cubic mm | Total Weight: 6.7670kg)
Bin 4: Amt. Items 14 (Total Volume: 74595451 cubic mm | Total Weight: 2.4480kg)
Bin 5: Amt. Items 19 (Total Volume: 74840593 cubic mm | Total Weight: 4.2340kg)
Bin 6: Amt. Items 7 (Total Volume: 75308492 cubic mm | Total Weight: 2.5050kg)
Bin 7: Amt. Items 19 (Total Volume: 75420310 cubic mm | Total Weight: 4.1400kg)
Bin 8: Amt. Items 9 (Total Volume: 74239108 cubic mm | Total Weight: 3.2200kg)
Bin 9: Amt. Items 13 (Total Volume: 74377824 cubic mm | Total Weight: 4.5420kg)
Bin 10: Amt. Items 14 (Total Volume: 74640852 cubic mm | Total Weight: 5.1380kg)
Bin 11: Amt. Items 12 (Total Volume: 75338845 cubic mm | Total Weight: 3.4730kg)
Bin 12: Amt. Items 12 (Total Volume: 75263039 cubic mm | Total Weight: 3.5500kg)
Bin 13: Amt. 

### 2.1.3- Situação 3

In [49]:
df_ffd_s3 = df_process_capacitado.query("chave_loja_buffer_onda in ('8_EN_4', '10_EN_4','3_EN_3')")
df_ffd_s3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33719,33719,6467,2023-10-26,539945948,285,338,71,0.140,1,CARTON_M,...,6839430,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6839430,0.140
33754,33754,102,2023-10-26,539945948,285,338,71,0.140,1,CARTON_M,...,6839430,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6839430,0.140
33858,33858,915,2023-10-30,847254621,445,242,33,0.323,2,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7107540,0.646
33859,33859,40,2023-10-30,847254655,443,233,32,0.303,2,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6606016,0.606
33860,33860,4051,2023-10-30,847254663,431,225,34,0.295,1,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,3297150,0.295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59486,59486,843,2023-11-21,861665901,496,364,19,0.300,6,CARTON_M,...,3430336,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,20582016,1.800
59487,59487,4116,2023-11-21,861665944,465,355,21,0.267,2,CARTON_M,...,3466575,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6933150,0.534
59488,59488,5980,2023-11-21,861665910,478,304,23,0.280,4,CARTON_M,...,3342176,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13368704,1.120
59489,59489,8861,2023-11-21,861665899,451,368,21,0.313,3,CARTON_M,...,3485328,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10455984,0.939


In [50]:
df_ffd_s3['Peças'].sum()

4532

In [51]:
df_ffd_s3['Volume_total'].sum()

21161399219

In [52]:
bin_capacity_vol = 75538008
bin_capacity_weight = 23
bin_item_limit = 512
packed_bins, leftovers = first_fit_decreasing(df_ffd_s3, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")

Packed bins (new method):
Bin 1: Amt. Items 20 (Total Volume: 75458526 cubic mm | Total Weight: 7.3600kg)
Bin 2: Amt. Items 23 (Total Volume: 75362040 cubic mm | Total Weight: 7.7170kg)
Bin 3: Amt. Items 19 (Total Volume: 75186105 cubic mm | Total Weight: 4.8070kg)
Bin 4: Amt. Items 17 (Total Volume: 75257553 cubic mm | Total Weight: 4.5730kg)
Bin 5: Amt. Items 14 (Total Volume: 75219294 cubic mm | Total Weight: 5.0800kg)
Bin 6: Amt. Items 19 (Total Volume: 75349756 cubic mm | Total Weight: 6.5560kg)
Bin 7: Amt. Items 13 (Total Volume: 74784992 cubic mm | Total Weight: 4.8930kg)
Bin 8: Amt. Items 28 (Total Volume: 74911604 cubic mm | Total Weight: 7.3710kg)
Bin 9: Amt. Items 15 (Total Volume: 75130825 cubic mm | Total Weight: 5.1100kg)
Bin 10: Amt. Items 14 (Total Volume: 75330336 cubic mm | Total Weight: 5.1450kg)
Bin 11: Amt. Items 13 (Total Volume: 75412188 cubic mm | Total Weight: 4.4750kg)
Bin 12: Amt. Items 14 (Total Volume: 74613428 cubic mm | Total Weight: 4.4600kg)
Bin 13: Amt

### 2.1.4- Situação 4

In [53]:
df_ffd_s4 = df_process_capacitado.query("chave_loja_buffer_onda in ('10_CX_1', '3_CX_9', '3_CX_1', '10_CX_4')")
df_ffd_s4

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.09
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.09
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.40
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.40
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,1375000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1375000,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32838,32838,9489,2023-11-16,882198071,290,258,34,0.350,3,CARTON_M,...,2543880,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,7631640,1.05
32839,32839,9951,2023-11-16,882198003,332,264,35,0.500,7,CARTON_M,...,3067680,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,21473760,3.50
32840,32840,10874,2023-11-16,882198054,321,263,96,0.400,9,CARTON_M,...,8104608,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,72941472,3.60
32841,32841,3749,2023-11-16,503100046,215,119,33,0.140,88,CARTON_M,...,844305,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,74298840,12.32


In [54]:
df_ffd_s4['Peças'].sum()

6634

In [55]:
df_ffd_s4['Volume_total'].sum()

15757146007

In [56]:
bin_capacity_vol = 83393894.4
bin_capacity_weight = 23
bin_item_limit = 170
packed_bins, leftovers = first_fit_decreasing(df_ffd_s4, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")

Packed bins (new method):
Bin 1: Amt. Items 44 (Total Volume: 83368971 cubic mm | Total Weight: 7.0660kg)
Bin 2: Amt. Items 49 (Total Volume: 83381956 cubic mm | Total Weight: 6.8580kg)
Bin 3: Amt. Items 66 (Total Volume: 83030748 cubic mm | Total Weight: 10.7070kg)
Bin 4: Amt. Items 15 (Total Volume: 83198471 cubic mm | Total Weight: 9.3690kg)
Bin 5: Amt. Items 41 (Total Volume: 83058870 cubic mm | Total Weight: 8.1940kg)
Bin 6: Amt. Items 29 (Total Volume: 82295637 cubic mm | Total Weight: 8.4650kg)
Bin 7: Amt. Items 20 (Total Volume: 83092780 cubic mm | Total Weight: 8.4070kg)
Bin 8: Amt. Items 26 (Total Volume: 82328392 cubic mm | Total Weight: 11.6620kg)
Bin 9: Amt. Items 26 (Total Volume: 83341294 cubic mm | Total Weight: 4.6500kg)
Bin 10: Amt. Items 25 (Total Volume: 83370910 cubic mm | Total Weight: 9.0730kg)
Bin 11: Amt. Items 28 (Total Volume: 82821393 cubic mm | Total Weight: 9.8410kg)
Bin 12: Amt. Items 27 (Total Volume: 82943980 cubic mm | Total Weight: 10.5380kg)
Bin 13: 

### 2.1.5- Situação 5

In [57]:
df_ffd_s5 = df_process_capacitado.query("chave_loja_buffer_onda in ('31_CX_1', '31_CX_9', '40_CX_9', '2_CX_9', '16_CX_6', '2_CX_4')")
df_ffd_s5

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
123,123,871,2023-10-24,534996762,364,167,90,0.090,2,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,10941840,0.180
124,124,1321,2023-10-24,534996754,359,146,76,0.080,4,CARTON_M,...,3983464,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,15933856,0.320
125,125,871,2023-10-24,552747022,250,100,9,0.108,1,CARTON_M,...,225000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,225000,0.108
126,126,1321,2023-10-24,539354355,250,100,70,0.200,1,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1750000,0.200
127,127,871,2023-10-24,539354339,250,100,58,0.175,1,CARTON_M,...,1450000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1450000,0.175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33012,33012,388,2023-11-16,883916193,302,213,60,0.500,2,CARTON_M,...,3859560,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,7719120,1.000
33013,33013,5958,2023-11-16,883916257,287,209,86,0.460,1,CARTON_M,...,5158538,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5158538,0.460
33014,33014,6693,2023-11-16,883916214,294,223,66,0.460,2,CARTON_M,...,4327092,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,8654184,0.920
33015,33015,8515,2023-11-16,883916441,289,211,74,0.400,1,CARTON_M,...,4512446,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,4512446,0.400


In [58]:
df_ffd_s5['Peças'].sum()

7593

In [59]:
df_ffd_s5['Volume_total'].sum()

15804355296

In [60]:
bin_capacity_vol = 83393894.4
bin_capacity_weight = 23
bin_item_limit = 170
packed_bins, leftovers = first_fit_decreasing(df_ffd_s5, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")

Packed bins (new method):
Bin 1: Amt. Items 42 (Total Volume: 83340714 cubic mm | Total Weight: 4.6510kg)
Bin 2: Amt. Items 38 (Total Volume: 83159032 cubic mm | Total Weight: 7.8090kg)
Bin 3: Amt. Items 43 (Total Volume: 83320296 cubic mm | Total Weight: 9.0770kg)
Bin 4: Amt. Items 36 (Total Volume: 83327634 cubic mm | Total Weight: 7.0650kg)
Bin 5: Amt. Items 43 (Total Volume: 82992240 cubic mm | Total Weight: 8.0600kg)
Bin 6: Amt. Items 49 (Total Volume: 83199700 cubic mm | Total Weight: 10.7780kg)
Bin 7: Amt. Items 74 (Total Volume: 83176528 cubic mm | Total Weight: 10.6500kg)
Bin 8: Amt. Items 41 (Total Volume: 83333433 cubic mm | Total Weight: 9.1740kg)
Bin 9: Amt. Items 69 (Total Volume: 83162090 cubic mm | Total Weight: 8.8820kg)
Bin 10: Amt. Items 88 (Total Volume: 83384521 cubic mm | Total Weight: 22.1530kg)
Bin 11: Amt. Items 32 (Total Volume: 83051022 cubic mm | Total Weight: 10.2840kg)
Bin 12: Amt. Items 56 (Total Volume: 83297597 cubic mm | Total Weight: 6.5010kg)
Bin 13:

## 2.2- MODELO EXATO

Vamos adicionar uma condição em que o tipo de buffer varia conforme a chave da loja

<p>O peso máximo é mesmo, indenpendente o tipo de buffer. Além disso, nesse momento iremos utilizar o volume padrão para cada buffer</p>

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i k}$, um volume $v_{i k}$, um múltiplo $inner_{i k}$ da quantidade de itens a serem selecionados uma demanda $d_{i k}$, para todo $i \in \{1,...,n\}$ e $k \in \{1,...,r\}$, e um número $m$ de mochilas (caixas) diferentes com capacidade volumétrica $cv_{k}$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np_{k}$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na caixa $j$, para a loja $k$.


In [61]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
param pecas_caixa:= floor(cp / maxVal_peso);

param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a loja k

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;
   
s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

### 2.2.1- Situação 1

In [62]:
df_process_situacao1 = df_process_capacitado.query('chave_loja_buffer_onda in ("16_EN_3","9_EN_4","33_EN_3")')
#('chave_loja_buffer_onda in ["16_EN_3","10_CX_1","33_EN_3"]')

df_process_situacao1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33868,33868,40,2023-10-30,847254655,443,233,32,0.303,3,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,9909024,0.909
33869,33869,915,2023-10-30,847254621,445,242,33,0.323,3,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10661310,0.969
33870,33870,4051,2023-10-30,847254663,431,225,34,0.295,2,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6594300,0.590
33871,33871,8788,2023-10-30,847254680,445,244,33,0.328,2,CARTON_M,...,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7166280,0.656
33872,33872,8438,2023-10-30,847254639,432,248,32,0.310,4,CARTON_M,...,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13713408,1.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59786,59786,1204,2023-11-21,879747286,321,146,45,0.240,5,CARTON_M,...,2108970,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10544850,1.200
59787,59787,4234,2023-11-21,879747315,309,156,65,0.203,2,CARTON_M,...,3133260,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6266520,0.406
59788,59788,8697,2023-11-21,879747294,314,134,41,0.230,7,CARTON_M,...,1725116,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12075812,1.610
59789,59789,9768,2023-11-21,879747307,320,147,42,0.217,3,CARTON_M,...,1975680,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5927040,0.651


In [64]:
df_process_situacao1['Peças_max'].unique()

array([512], dtype=int64)

In [65]:
df_process_situacao1.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'prioridade', 'volume_util_caixa_Caixa_1', 'volume_util_caixa_Caixa_2',
       'volume_util_caixa_Caixa_3', 'volume_util_padrao', 'Peças_max',
       'Volume_total', 'Peso_total'],
      dtype='object')

#### 2.2.1.1- Entrada de dados no modelo

In [66]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,0.220,0.207,0.303
1,2,0.167,0.213,0.323
2,3,0.240,0.156,0.295
3,4,0.213,0.150,0.328
4,5,0.185,0.455,0.310
...,...,...,...,...
405,406,0.000,0.000,0.240
406,407,0.000,0.000,0.203
407,408,0.000,0.000,0.230
408,409,0.000,0.000,0.217


In [67]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,3001320.0,3228148.0,3303008.0
1,2,2741220.0,2456160.0,3553770.0
2,3,2864650.0,1212300.0,3297150.0
3,4,2456160.0,2290431.0,3583140.0
4,5,2637180.0,4548648.0,3428352.0
...,...,...,...,...
405,406,0.0,0.0,2108970.0
406,407,0.0,0.0,3133260.0
407,408,0.0,0.0,1725116.0
408,409,0.0,0.0,1975680.0


In [70]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
405,406,0.0,0.0,1.0
406,407,0.0,0.0,1.0
407,408,0.0,0.0,1.0
408,409,0.0,0.0,1.0


In [71]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,3.0
1,2,1.0,1.0,3.0
2,3,4.0,1.0,2.0
3,4,2.0,2.0,2.0
4,5,1.0,1.0,4.0
...,...,...,...,...
405,406,0.0,0.0,5.0
406,407,0.0,0.0,2.0
407,408,0.0,0.0,7.0
408,409,0.0,0.0,3.0


Capacidade da caixa como variável

In [72]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao1.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,16_EN_3,512.0
1,33_EN_3,512.0
2,9_EN_4,512.0


In [73]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao1.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,16_EN_3,75538008.0
1,33_EN_3,75538008.0
2,9_EN_4,75538008.0


Características das peças

In [74]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

Características da caixa

In [75]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [76]:
cv['volume_util_padrao'].values

array([75538008., 75538008., 75538008.])

In [77]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

In [78]:
ampl.getValue('soma')

5343

In [79]:
ampl.getValue('maxbins')

268

In [80]:
ampl.getValue('pecas_caixa')

20

#### 2.2.1.2- Solvers

##### Solver Highs

In [81]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 3e+01]
Presolving model
3453 rows, 279792 cols, 1118364 nonzeros  0s
3453 rows, 279792 cols, 1118364 nonzeros  5s
Objective function is integral with scale 1

Solving MIP model with:
   3453 rows
   279792 cols (72360 binary, 207432 integer, 0 implied int., 0 continuous)
   1118364 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     6.0s
 R       0       0         0   0.00%   322.6013599     431               25.15%        0     

In [82]:
%%ampl_eval
display d, w, v, inner;

:       d      w        v     inner    :=
1   1    1   0.22     3001320    1
1   2    1   0.207    3228150    1
1   3    3   0.303    3303010    1
2   1    1   0.167    2741220    1
2   2    1   0.213    2456160    1
2   3    3   0.323    3553770    1
3   1    4   0.24     2864650    1
3   2    1   0.156    1212300    1
3   3    2   0.295    3297150    1
4   1    2   0.213    2456160    1
4   2    2   0.15     2290430    1
4   3    2   0.328    3583140    1
5   1    1   0.185    2637180    1
5   2    1   0.455    4548650    1
5   3    4   0.31     3428350    1
6   1    1   0.18     2668050    1
6   2    1   0.437    5118140    1
6   3    6   0.303    2701220    1
7   1    4   0.193    2458150    1
7   2    1   0.46     6075000    1
7   3    3   0.43     2957620    1
8   1    1   0.14     1049360    1
8   2    1   0.175    3433490    1
8   3    2   0.47     3426520    1
9   1    1   0.455    4548650    1
9   2    3   0.205    4291800    1
9   3    1   0.44     3200700    1
10  1    1   

In [83]:
e_s1_highs = ampl.getVariable('e').getValues().toPandas()
e_s1_highs = e_s1_highs.reset_index()
e_s1_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1_highs = e_s1_highs.query('valor_multiplo >0')
e_s1_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
53,1,18,3,2.0
271,1,91,2,1.0
401,1,134,3,1.0
507,1,170,1,1.0
1270,2,156,2,1.0
...,...,...,...,...
326999,407,192,3,1.0
327755,408,176,3,1.0
327947,408,240,3,6.0
328112,409,27,3,3.0


In [84]:
df_agrupamento_loja1_highs_s1 = e_s1_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s1 = df_agrupamento_loja1_highs_s1.reset_index()
df_agrupamento_loja1_highs_s1.columns.name = None
df_agrupamento_loja1_highs_s1 # as colunas são as caixas

,item_loja,1,2,4,13,16,18,20,22,23,...,247,249,257,259,260,261,262,263,266,267
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao1.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [86]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [87]:
df_agrupamento_loja1_highs_s1['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s1['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s1

,item_loja,1,2,4,13,16,18,20,22,23,...,257,259,260,261,262,263,266,267,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34013,16_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34014,16_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,34015,16_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34016,16_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34017,16_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57776,16_EN_3
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57777,16_EN_3
308,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57778,16_EN_3
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57779,16_EN_3


In [88]:
df_agrupamento_loja2_highs_s1 = e_s1_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s1 = df_agrupamento_loja2_highs_s1.reset_index()
df_agrupamento_loja2_highs_s1.columns.name = None
df_agrupamento_loja2_highs_s1['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s1['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s1

,item_loja,1,7,8,9,10,11,12,13,14,...,245,246,254,256,259,263,264,265,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34201,33_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34202,33_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34203,33_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34204,33_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37205,33_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58412,33_EN_3
316,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58413,33_EN_3
317,318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58414,33_EN_3
318,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58415,33_EN_3


In [89]:
y_s1_highs = ampl.getVariable('y').getValues().toPandas()
y_s1_highs = y_s1_highs.reset_index()
y_s1_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s1_highs = y_s1_highs.query('selecao>0')

df_caixas_por_loja_highs_s1 = y_s1_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s1 = df_caixas_por_loja_highs_s1.reset_index()
df_caixas_por_loja_highs_s1.columns.name = None
df_caixas_por_loja_highs_s1 # as colunas são as lojas

,Caixa,1,2,3
0,1,1.0,1.0,0.0
1,2,1.0,0.0,0.0
2,4,1.0,0.0,0.0
3,7,0.0,1.0,0.0
4,8,0.0,1.0,1.0
...,...,...,...,...
206,264,0.0,1.0,0.0
207,265,0.0,1.0,1.0
208,266,1.0,0.0,0.0
209,267,1.0,0.0,1.0


In [90]:
df_caixas_por_loja_highs_s1.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s1

,caixas,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,0.0
1,2,1.0,0.0,0.0
2,4,1.0,0.0,0.0
3,7,0.0,1.0,0.0
4,8,0.0,1.0,1.0
...,...,...,...,...
206,264,0.0,1.0,0.0
207,265,0.0,1.0,1.0
208,266,1.0,0.0,0.0
209,267,1.0,0.0,1.0


##### Caixas

In [91]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s1[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s1[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "16_EN_3": 119.0
número de caixas para a chave_loja_buffer_onda "33_EN_3": 119.0
número de caixas para a chave_loja_buffer_onda "9_EN_4": 96.0

total de caixas: 334.0


### 2.2.2- Situação 2

In [62]:
df_process_situacao2 = df_ffd_s2

df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33800,33800,40,2023-10-30,847254655,443,233,32,0.303,10,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,33030080,3.030
33801,33801,915,2023-10-30,847254621,445,242,33,0.323,10,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,35537700,3.230
33802,33802,4051,2023-10-30,847254663,431,225,34,0.295,6,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,19782900,1.770
33803,33803,8438,2023-10-30,847254639,432,248,32,0.310,12,CARTON_M,...,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,41140224,3.720
33804,33804,8788,2023-10-30,847254680,445,244,33,0.328,5,CARTON_M,...,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17915700,1.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,45989424,5.160
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,28047600,2.805
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6334200,0.574
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5670000,0.614


#### 2.2.2.1- Entrada de dados no modelo

In [63]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_EN_3,28_EN_4,2_EN_3
0,1,0.210,0.303,0.210
1,2,0.140,0.323,0.210
2,3,0.390,0.295,0.193
3,4,0.433,0.310,0.140
4,5,0.310,0.328,0.100
...,...,...,...,...
573,574,0.250,0.000,0.000
574,575,0.250,0.000,0.000
575,576,0.320,0.000,0.000
576,577,0.288,0.000,0.000


In [64]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_EN_3,28_EN_4,2_EN_3
0,1,4773600.0,3303008.0,4773600.0
1,2,1049364.0,3553770.0,4773600.0
2,3,4320315.0,3297150.0,1009120.0
3,4,6020820.0,3428352.0,1049364.0
4,5,3157578.0,3583140.0,1796535.0
...,...,...,...,...
573,574,1534522.0,0.0,0.0
574,575,1534522.0,0.0,0.0
575,576,2370060.0,0.0,0.0
576,577,2716025.0,0.0,0.0


In [65]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_EN_3,28_EN_4,2_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
573,574,1.0,0.0,0.0
574,575,1.0,0.0,0.0
575,576,1.0,0.0,0.0
576,577,1.0,0.0,0.0


In [66]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_EN_3,28_EN_4,2_EN_3
0,1,1.0,10.0,1.0
1,2,1.0,10.0,1.0
2,3,1.0,6.0,1.0
3,4,1.0,12.0,1.0
4,5,1.0,5.0,5.0
...,...,...,...,...
573,574,5.0,0.0,0.0
574,575,9.0,0.0,0.0
575,576,11.0,0.0,0.0
576,577,6.0,0.0,0.0


Capacidade da caixa como variável

In [67]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_EN_3,512.0
1,28_EN_4,512.0
2,2_EN_3,512.0


In [68]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_EN_3,75538008.0
1,28_EN_4,75538008.0
2,2_EN_3,75538008.0


In [69]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [70]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [71]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.2.2- Solvers

In [72]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 4e+01]
Presolving model
4404 rows, 464688 cols, 1857744 nonzeros  0s
4404 rows, 464688 cols, 1857744 nonzeros  12s
Objective function is integral with scale 1

Solving MIP model with:
   4404 rows
   464688 cols (82992 binary, 381696 integer, 0 implied int., 0 continuous)
   1857744 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0    13.7s
 R       0       0         0   0.00%   556.6223966     716               22.26%        0    

In [78]:
e_s2_highs = ampl.getVariable('e').getValues().toPandas()
e_s2_highs = e_s2_highs.reset_index()
e_s2_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_highs = e_s2_highs.query('valor_multiplo >0')
e_s2_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
676,1,226,2,10
759,1,254,1,1
863,1,288,3,1
1054,2,16,2,1
1462,2,152,2,9
...,...,...,...,...
578979,575,130,1,9
579951,576,118,1,11
581082,577,159,1,6
582441,578,276,1,8


In [79]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

,Caixa,1,2,3
0,2,1.0,0.0,1.0
1,3,1.0,0.0,1.0
2,4,1.0,0.0,0.0
3,5,1.0,1.0,1.0
4,6,1.0,1.0,0.0
...,...,...,...,...
323,332,1.0,0.0,1.0
324,333,1.0,1.0,0.0
325,334,1.0,0.0,0.0
326,335,1.0,1.0,0.0


In [80]:
df_caixas_por_loja_highs_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s2

,caixas,10_EN_3,28_EN_4,2_EN_3
0,2,1.0,0.0,1.0
1,3,1.0,0.0,1.0
2,4,1.0,0.0,0.0
3,5,1.0,1.0,1.0
4,6,1.0,1.0,0.0
...,...,...,...,...
323,332,1.0,0.0,1.0
324,333,1.0,1.0,0.0
325,334,1.0,0.0,0.0
326,335,1.0,1.0,0.0


#### Caixas

In [81]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s2[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s2[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_EN_3": 273.0
número de caixas para a chave_loja_buffer_onda "28_EN_4": 268.0
número de caixas para a chave_loja_buffer_onda "2_EN_3": 174.0

total de caixas: 715.0


### 2.2.3- Situação 3

In [82]:
df_process_situacao3 = df_ffd_s3

df_process_situacao3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33719,33719,6467,2023-10-26,539945948,285,338,71,0.140,1,CARTON_M,...,6839430,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6839430,0.140
33754,33754,102,2023-10-26,539945948,285,338,71,0.140,1,CARTON_M,...,6839430,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6839430,0.140
33858,33858,915,2023-10-30,847254621,445,242,33,0.323,2,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7107540,0.646
33859,33859,40,2023-10-30,847254655,443,233,32,0.303,2,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6606016,0.606
33860,33860,4051,2023-10-30,847254663,431,225,34,0.295,1,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,3297150,0.295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59486,59486,843,2023-11-21,861665901,496,364,19,0.300,6,CARTON_M,...,3430336,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,20582016,1.800
59487,59487,4116,2023-11-21,861665944,465,355,21,0.267,2,CARTON_M,...,3466575,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6933150,0.534
59488,59488,5980,2023-11-21,861665910,478,304,23,0.280,4,CARTON_M,...,3342176,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13368704,1.120
59489,59489,8861,2023-11-21,861665899,451,368,21,0.313,3,CARTON_M,...,3485328,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10455984,0.939


#### 2.2.3.1- Entrada de dados no modelo

In [83]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_EN_4,3_EN_3,8_EN_4
0,1,0.140,0.180,0.140
1,2,0.380,0.140,0.323
2,3,0.325,0.110,0.303
3,4,0.360,0.437,0.295
4,5,0.360,0.287,0.310
...,...,...,...,...
505,506,0.000,0.000,0.300
506,507,0.000,0.000,0.267
507,508,0.000,0.000,0.280
508,509,0.000,0.000,0.313


In [84]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_EN_4,3_EN_3,8_EN_4
0,1,6839430.0,2668050.0,6839430.0
1,2,3448182.0,1049364.0,3553770.0
2,3,3448182.0,1221480.0,3303008.0
3,4,2643030.0,5118144.0,3297150.0
4,5,2434600.0,4271909.0,3428352.0
...,...,...,...,...
505,506,0.0,0.0,3430336.0
506,507,0.0,0.0,3466575.0
507,508,0.0,0.0,3342176.0
508,509,0.0,0.0,3485328.0


In [85]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_EN_4,3_EN_3,8_EN_4
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
505,506,0.0,0.0,1.0
506,507,0.0,0.0,1.0
507,508,0.0,0.0,1.0
508,509,0.0,0.0,1.0


In [86]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_EN_4,3_EN_3,8_EN_4
0,1,1.0,1.0,1.0
1,2,2.0,1.0,2.0
2,3,1.0,4.0,2.0
3,4,5.0,1.0,1.0
4,5,6.0,2.0,2.0
...,...,...,...,...
505,506,0.0,0.0,6.0
506,507,0.0,0.0,2.0
507,508,0.0,0.0,4.0
508,509,0.0,0.0,3.0


Capacidade da caixa como variável

In [87]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_EN_4,512.0
1,3_EN_3,512.0
2,8_EN_4,512.0


In [88]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_EN_4,75538008.0
1,3_EN_3,75538008.0
2,8_EN_4,75538008.0


In [89]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [90]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [91]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.3.2- Solvers

In [92]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 2e+01]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities    816    4.717e+07    2.266e+08
Row     residuals            0            0            0
Attempting to find feasible solution for (partial) user-supplied values of discrete variables
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+01]
  RHS    [1e+00, 2e+01]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time    

In [93]:
e_s3_highs = ampl.getVariable('e').getValues().toPandas()
e_s3_highs = e_s3_highs.reset_index()
e_s3_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s3_highs = e_s3_highs.query('valor_multiplo >0')
e_s3_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
68,1,23,3,1.0
151,1,51,2,1.0
387,1,130,1,1.0
701,2,18,3,2.0
1134,2,163,1,2.0
...,...,...,...,...
329303,509,40,3,2.0
329307,509,42,1,4.0
329543,509,120,3,1.0
329951,510,40,3,2.0


In [94]:
y_s3_highs = ampl.getVariable('y').getValues().toPandas()
y_s3_highs = y_s3_highs.reset_index()
y_s3_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s3_highs = y_s3_highs.query('selecao>0')

df_caixas_por_loja_highs_s3 = y_s3_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s3 = df_caixas_por_loja_highs_s3.reset_index()
df_caixas_por_loja_highs_s3.columns.name = None
df_caixas_por_loja_highs_s3 # as colunas são as lojas

,Caixa,1,2,3
0,1,1.0,1.0,1.0
1,2,0.0,1.0,1.0
2,3,0.0,1.0,0.0
3,4,1.0,1.0,0.0
4,5,0.0,0.0,1.0
...,...,...,...,...
172,208,1.0,1.0,1.0
173,210,1.0,1.0,0.0
174,211,0.0,1.0,1.0
175,214,1.0,0.0,0.0


In [95]:
df_caixas_por_loja_highs_s3.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s3

,caixas,10_EN_4,3_EN_3,8_EN_4
0,1,1.0,1.0,1.0
1,2,0.0,1.0,1.0
2,3,0.0,1.0,0.0
3,4,1.0,1.0,0.0
4,5,0.0,0.0,1.0
...,...,...,...,...
172,208,1.0,1.0,1.0
173,210,1.0,1.0,0.0
174,211,0.0,1.0,1.0
175,214,1.0,0.0,0.0


#### Caixas

In [96]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s3[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s3[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_EN_4": 98.99999999999989
número de caixas para a chave_loja_buffer_onda "3_EN_3": 85.0
número de caixas para a chave_loja_buffer_onda "8_EN_4": 103.0

total de caixas: 286.9999999999999


### 2.2.4- Situação 4

In [117]:
df_process_situacao4 = df_ffd_s4

df_process_situacao4

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total,item_loja
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.09,1
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.09,2
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.40,3
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.40,4
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1375000,0.18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32838,32838,9489,2023-11-16,882198071,290,258,34,0.350,3,CARTON_M,...,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,7631640,1.05,167
32839,32839,9951,2023-11-16,882198003,332,264,35,0.500,7,CARTON_M,...,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,21473760,3.50,168
32840,32840,10874,2023-11-16,882198054,321,263,96,0.400,9,CARTON_M,...,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,72941472,3.60,169
32841,32841,3749,2023-11-16,503100046,215,119,33,0.140,88,CARTON_M,...,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,74298840,12.32,170


#### 2.2.4.1- Entrada de dados no modelo

In [99]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_CX_1,10_CX_4,3_CX_1,3_CX_9
0,1,0.090,0.203,0.080,0.220
1,2,0.090,0.203,0.108,0.153
2,3,0.200,0.203,0.220,0.230
3,4,0.200,0.203,0.200,0.230
4,5,0.180,0.203,0.200,0.187
...,...,...,...,...,...
399,400,0.027,0.000,0.000,0.000
400,401,0.147,0.000,0.000,0.000
401,402,0.137,0.000,0.000,0.000
402,403,0.143,0.000,0.000,0.000


In [100]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_CX_1,10_CX_4,3_CX_1,3_CX_9
0,1,5470920.0,2800000.0,3983464.0,1556100.0
1,2,5470920.0,2000000.0,225000.0,1681500.0
2,3,1750000.0,2800000.0,3777750.0,1430100.0
3,4,1750000.0,2000000.0,1750000.0,2275200.0
4,5,1375000.0,2800000.0,1750000.0,2096496.0
...,...,...,...,...,...
399,400,670670.0,0.0,0.0,0.0
400,401,3555576.0,0.0,0.0,0.0
401,402,3482136.0,0.0,0.0,0.0
402,403,3961375.0,0.0,0.0,0.0


In [101]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_CX_1,10_CX_4,3_CX_1,3_CX_9
0,1,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0
...,...,...,...,...,...
399,400,1.0,0.0,0.0,0.0
400,401,1.0,0.0,0.0,0.0
401,402,1.0,0.0,0.0,0.0
402,403,1.0,0.0,0.0,0.0


In [102]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_CX_1,10_CX_4,3_CX_1,3_CX_9
0,1,1.0,4.0,1.0,7.0
1,2,1.0,6.0,1.0,6.0
2,3,2.0,5.0,4.0,17.0
3,4,2.0,11.0,3.0,4.0
4,5,1.0,15.0,2.0,12.0
...,...,...,...,...,...
399,400,4.0,0.0,0.0,0.0
400,401,9.0,0.0,0.0,0.0
401,402,6.0,0.0,0.0,0.0
402,403,9.0,0.0,0.0,0.0


Capacidade da caixa como variável

In [104]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao4.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_CX_1,170.0
1,10_CX_4,170.0
2,3_CX_1,170.0
3,3_CX_9,170.0


In [105]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao4.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_CX_1,83393894.4
1,10_CX_4,83393894.4
2,3_CX_1,83393894.4
3,3_CX_9,83393894.4


In [106]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [107]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [108]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.4.2- Solvers

In [109]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [5e-15, 4e+01]
  RHS    [1e+00, 9e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.09 (total)
                    0.06 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 4e+01]
  RHS    [1e+00, 9e+01]
Presolving model
4642 rows, 340442 cols, 1360612 nonzeros  0s
4642 rows, 340442 cols, 1360612 nonzeros  9s
Objective function is integral with scale 1

Solving MIP model with:
   4642 row

In [113]:
e_s4_highs = ampl.getVariable('e').getValues().toPandas()
e_s4_highs = e_s4_highs.reset_index()
e_s4_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s4_highs = e_s4_highs.query('valor_multiplo >0')
e_s4_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
279,1,70,4,4
381,1,96,2,3
689,1,173,2,1
864,1,217,1,1
1074,1,269,3,1
...,...,...,...,...
465784,403,269,1,6
466276,404,103,1,2
466318,404,113,3,2
466340,404,119,1,3


In [114]:
y_s4_highs = ampl.getVariable('y').getValues().toPandas()
y_s4_highs = y_s4_highs.reset_index()
y_s4_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s4_highs = y_s4_highs.query('selecao>0')

df_caixas_por_loja_highs_s4 = y_s4_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s4 = df_caixas_por_loja_highs_s4.reset_index()
df_caixas_por_loja_highs_s4.columns.name = None
df_caixas_por_loja_highs_s4 # as colunas são as lojas

,Caixa,1,2,3,4
0,1,1.0,1.0,1.0,0.0
1,4,1.0,1.0,0.0,1.0
2,5,0.0,0.0,0.0,1.0
3,6,0.0,1.0,0.0,1.0
4,8,1.0,1.0,1.0,0.0
...,...,...,...,...,...
185,284,0.0,1.0,0.0,0.0
186,285,0.0,0.0,1.0,0.0
187,287,0.0,0.0,1.0,0.0
188,288,1.0,1.0,0.0,0.0


In [115]:
df_caixas_por_loja_highs_s4.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s4

,caixas,10_CX_1,10_CX_4,3_CX_1,3_CX_9
0,1,1.0,1.0,1.0,0.0
1,4,1.0,1.0,0.0,1.0
2,5,0.0,0.0,0.0,1.0
3,6,0.0,1.0,0.0,1.0
4,8,1.0,1.0,1.0,0.0
...,...,...,...,...,...
185,284,0.0,1.0,0.0,0.0
186,285,0.0,0.0,1.0,0.0
187,287,0.0,0.0,1.0,0.0
188,288,1.0,1.0,0.0,0.0


#### Caixas

In [116]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s4[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s4[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_CX_1": 68.0
número de caixas para a chave_loja_buffer_onda "10_CX_4": 91.0
número de caixas para a chave_loja_buffer_onda "3_CX_1": 59.0
número de caixas para a chave_loja_buffer_onda "3_CX_9": 71.0

total de caixas: 289.0


### 2.2.5- Situação 5

In [119]:
df_process_situacao5 = df_ffd_s5

df_process_situacao5

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
123,123,871,2023-10-24,534996762,364,167,90,0.090,2,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,10941840,0.180
124,124,1321,2023-10-24,534996754,359,146,76,0.080,4,CARTON_M,...,3983464,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,15933856,0.320
125,125,871,2023-10-24,552747022,250,100,9,0.108,1,CARTON_M,...,225000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,225000,0.108
126,126,1321,2023-10-24,539354355,250,100,70,0.200,1,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1750000,0.200
127,127,871,2023-10-24,539354339,250,100,58,0.175,1,CARTON_M,...,1450000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1450000,0.175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33012,33012,388,2023-11-16,883916193,302,213,60,0.500,2,CARTON_M,...,3859560,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,7719120,1.000
33013,33013,5958,2023-11-16,883916257,287,209,86,0.460,1,CARTON_M,...,5158538,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5158538,0.460
33014,33014,6693,2023-11-16,883916214,294,223,66,0.460,2,CARTON_M,...,4327092,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,8654184,0.920
33015,33015,8515,2023-11-16,883916441,289,211,74,0.400,1,CARTON_M,...,4512446,6,78,27778950.0,83393894.4,75538008.0,83393894.4,170,4512446,0.400


#### 2.2.5.1- Entrada de dados no modelo

In [120]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,16_CX_6,2_CX_4,2_CX_9,31_CX_1,31_CX_9,40_CX_9
0,1,0.49,0.727,0.220,0.090,0.220,0.220
1,2,0.24,0.670,0.230,0.080,0.153,0.220
2,3,0.35,0.653,0.230,0.108,0.230,0.230
3,4,0.26,0.500,0.187,0.200,0.230,0.230
4,5,0.06,0.740,0.153,0.175,0.230,0.187
...,...,...,...,...,...,...,...
270,271,0.00,0.530,0.000,0.000,0.000,0.000
271,272,0.00,0.290,0.000,0.000,0.000,0.000
272,273,0.00,0.267,0.000,0.000,0.000,0.000
273,274,0.00,0.273,0.000,0.000,0.000,0.000


In [121]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,16_CX_6,2_CX_4,2_CX_9,31_CX_1,31_CX_9,40_CX_9
0,1,820080.0,4995000.0,1556100.0,5470920.0,1556100.0,2214000.0
1,2,1506720.0,6213000.0,1430100.0,3983464.0,1681500.0,1556100.0
2,3,261690.0,5716386.0,2275200.0,225000.0,1430100.0,1430100.0
3,4,134995.0,3840000.0,2096496.0,1750000.0,1430100.0,2275200.0
4,5,250000.0,6057744.0,1681500.0,1450000.0,2275200.0,2096496.0
...,...,...,...,...,...,...,...
270,271,0.0,2081700.0,0.0,0.0,0.0,0.0
271,272,0.0,4727730.0,0.0,0.0,0.0,0.0
272,273,0.0,1200000.0,0.0,0.0,0.0,0.0
273,274,0.0,1548800.0,0.0,0.0,0.0,0.0


In [122]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,16_CX_6,2_CX_4,2_CX_9,31_CX_1,31_CX_9,40_CX_9
0,1,1.0,1.0,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0,1.0
4,5,6.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
270,271,0.0,1.0,0.0,0.0,0.0,0.0
271,272,0.0,1.0,0.0,0.0,0.0,0.0
272,273,0.0,1.0,0.0,0.0,0.0,0.0
273,274,0.0,1.0,0.0,0.0,0.0,0.0


In [123]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,16_CX_6,2_CX_4,2_CX_9,31_CX_1,31_CX_9,40_CX_9
0,1,8.0,1.0,6.0,2.0,4.0,1.0
1,2,1.0,1.0,15.0,4.0,4.0,3.0
2,3,3.0,1.0,5.0,1.0,1.0,11.0
3,4,4.0,1.0,4.0,1.0,8.0,31.0
4,5,12.0,1.0,6.0,1.0,16.0,21.0
...,...,...,...,...,...,...,...
270,271,0.0,2.0,0.0,0.0,0.0,0.0
271,272,0.0,6.0,0.0,0.0,0.0,0.0
272,273,0.0,4.0,0.0,0.0,0.0,0.0
273,274,0.0,6.0,0.0,0.0,0.0,0.0


Capacidade da caixa como variável

In [124]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao5.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,16_CX_6,170.0
1,2_CX_4,170.0
2,2_CX_9,170.0
3,31_CX_1,170.0
4,31_CX_9,170.0
5,40_CX_9,170.0


In [125]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao5.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,16_CX_6,83393894.4
1,2_CX_4,83393894.4
2,2_CX_9,83393894.4
3,31_CX_1,83393894.4
4,31_CX_9,83393894.4
5,40_CX_9,83393894.4


In [126]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [127]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [128]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.5.2- Solvers

In [129]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [9e-03, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 5e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.19 (total)
                    0.16 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [9e-03, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 5e+01]
Presolving model
7391 rows, 404474 cols, 1615820 nonzeros  0s
7391 rows, 404474 cols, 1615820 nonzeros  17s
Objective function is integral with scale 1

Solving MIP model with:
   7391 ro

In [130]:
e_s5_highs = ampl.getVariable('e').getValues().toPandas()
e_s5_highs = e_s5_highs.reset_index()
e_s5_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s5_highs = e_s5_highs.query('valor_multiplo >0')
e_s5_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
236,1,40,3,1
406,1,68,5,4
558,1,94,1,8
1532,1,256,3,5
1701,1,284,4,2
...,...,...,...,...
568230,274,248,1,5
568789,274,341,2,1
569426,275,101,3,3
569911,275,182,2,4


In [131]:
y_s5_highs = ampl.getVariable('y').getValues().toPandas()
y_s5_highs = y_s5_highs.reset_index()
y_s5_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s5_highs = y_s5_highs.query('selecao>0')

df_caixas_por_loja_highs_s5 = y_s5_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s5 = df_caixas_por_loja_highs_s5.reset_index()
df_caixas_por_loja_highs_s5.columns.name = None
df_caixas_por_loja_highs_s5 # as colunas são as lojas

,Caixa,1,2,3,4,5,6
0,1,0.0,0.0,0.0,1.0,0.0,0.0
1,2,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,1.0,1.0,0.0,1.0
3,5,0.0,0.0,1.0,0.0,0.0,0.0
4,6,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
224,341,0.0,1.0,0.0,1.0,0.0,0.0
225,342,0.0,1.0,1.0,0.0,0.0,0.0
226,343,0.0,1.0,0.0,0.0,0.0,0.0
227,344,0.0,1.0,0.0,0.0,1.0,0.0


In [132]:
df_caixas_por_loja_highs_s5.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s5

,caixas,16_CX_6,2_CX_4,2_CX_9,31_CX_1,31_CX_9,40_CX_9
0,1,0.0,0.0,0.0,1.0,0.0,0.0
1,2,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,1.0,1.0,0.0,1.0
3,5,0.0,0.0,1.0,0.0,0.0,0.0
4,6,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
224,341,0.0,1.0,0.0,1.0,0.0,0.0
225,342,0.0,1.0,1.0,0.0,0.0,0.0
226,343,0.0,1.0,0.0,0.0,0.0,0.0
227,344,0.0,1.0,0.0,0.0,1.0,0.0


#### Caixas

In [133]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s5[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s5[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "16_CX_6": 24.0
número de caixas para a chave_loja_buffer_onda "2_CX_4": 65.0
número de caixas para a chave_loja_buffer_onda "2_CX_9": 52.0
número de caixas para a chave_loja_buffer_onda "31_CX_1": 62.0
número de caixas para a chave_loja_buffer_onda "31_CX_9": 56.0
número de caixas para a chave_loja_buffer_onda "40_CX_9": 46.0

total de caixas: 305.0
